In [1]:
%matplotlib inline

### use vgg16 model , do not include output layer .original output is 1000 and here want the new output is 1

In [2]:
from tensorflow.python.keras.applications.vgg16 import VGG16
vgg16 = VGG16(include_top = False, input_shape=(224,224,3))
vgg16.summary()

58892288/58889256 [==============================]58892288/58889256 [==============================] - 40s 1us/step

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________

### modify the model and add new output layer for new output. Use sequential to create new model

In [7]:
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense,Dropout, Flatten
def build_transfer_model(vgg16):
    # transfer to sequential model and no trainable
    model = Sequential(vgg16.layers)
    print(len(model.layers))
    for layer in model.layers[:15]:
        layer.trainable = False
    #add new output layer
    model.add(Flatten())
    model.add(Dense(256,activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1,activation='sigmoid'))
    
    return model
model = build_transfer_model(vgg16)

19


### compile new model

In [8]:
from tensorflow.python.keras.optimizers import SGD
model.compile(
    loss = 'binary_crossentropy',
    optimizer = SGD(lr=1e-4,momentum = 0.9),
    metrics = ['accuracy']
)


Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [9]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

### can use generator to create mini batch data

In [10]:
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.applications.vgg16 import preprocess_input
## change data scale 
idg_train = ImageDataGenerator(
    rescale = 1/255.,
    shear_range = 0.1,
    zoom_range = 0.1,
    horizontal_flip = True,
    preprocessing_function = preprocess_input
)

### create iterator to load image

In [11]:
# for train data(@ learning)
img_itr_train = idg_train.flow_from_directory(
    'img/shrine_temple/train',
    target_size = (224,224),
    batch_size = 16,
    class_mode = 'binary'
)
# for confirm data(@ learning)
img_itr_validation = idg_train.flow_from_directory(
    'img/shrine_temple/validation',
    target_size = (224,224),
    batch_size = 16,
    class_mode = 'binary'
)

Found 600 images belonging to 2 classes.
Found 200 images belonging to 2 classes.


### output directory setting 

In [12]:
import os
from datetime import datetime
model_dir = os.path.join(
    'models',
    datetime.now().strftime('%y%m%d_%H%M')
)
os.makedirs(model_dir, exist_ok = True)
print('model_dir:',model_dir)
dir_weights = os.path.join(model_dir,'weights')
os.makedirs(dir_weights,exist_ok = True)
print('dir_weights:',dir_weights)

model_dir: models\180913_1408
dir_weights: models\180913_1408\weights


### save network and output label information

In [17]:
import json
import pickle
# save network
model_json = os.path.join(model_dir, 'model.json')
with open(model_json,'w') as f:    #write only
    json.dump(model.to_json(),f)

# save true label
model_classes = os.path.join(model_dir,'classes.pkl')
with open(model_classes, 'wb') as f:   #write only for binary 
    pickle.dump(img_itr_train.class_indices, f) # save label info
print(img_itr_train.class_indices)

{'shrine': 0, 'temple': 1}


### calculate how many batch is necessary for 1 epoch

In [18]:
import math
batch_size = 16
steps_per_epoch = math.ceil(img_itr_train.samples/batch_size)
validation_steps = math.ceil(img_itr_validation.samples/batch_size)
print (img_itr_train.samples)

600


### set callback

In [19]:
from tensorflow.python.keras.callbacks import ModelCheckpoint, CSVLogger
# Callback setting
cp_filepath = os.path.join(dir_weights, 'ep_{epoch:02d}_ls_{loss:.1f}.h5')
cp = ModelCheckpoint(
    cp_filepath,
    monitor = 'loss',
    verbose = 0,
    save_best_only = False,
    save_weights_only = True,
    mode = 'auto',
    period = 5
)
csv_filepath = os.path.join(model_dir, 'loss.csv')
csv = CSVLogger(csv_filepath, append = True)

### model learning start

In [21]:
n_epoch = 20
history = model.fit_generator(
    img_itr_train,
    steps_per_epoch = steps_per_epoch,
    epochs = n_epoch,
    validation_data = img_itr_validation,
    validation_steps = validation_steps,
    callbacks = [cp,csv]
)

Epoch 1/20
38/38 [==============================]38/38 [==============================] - 434s 11s/step - loss: 0.3644 - acc: 0.8470 - val_loss: 0.2474 - val_acc: 0.9150

Epoch 2/20
38/38 [==============================]38/38 [==============================] - 432s 11s/step - loss: 0.3350 - acc: 0.8618 - val_loss: 0.2147 - val_acc: 0.9300

Epoch 3/20
38/38 [==============================]38/38 [==============================] - 431s 11s/step - loss: 0.3097 - acc: 0.8717 - val_loss: 0.2061 - val_acc: 0.9250

Epoch 4/20
38/38 [==============================]38/38 [==============================] - 430s 11s/step - loss: 0.2877 - acc: 0.8832 - val_loss: 0.2441 - val_acc: 0.8800

Epoch 5/20
38/38 [==============================]38/38 [==============================] - 433s 11s/step - loss: 0.2619 - acc: 0.8947 - val_loss: 0.1949 - val_acc: 0.9250

Epoch 6/20
38/38 [==============================]38/38 [==============================] - 441s 12s/step - loss: 0.2384 - acc: 0.9062 - val_loss: 

### check the predict result

In [31]:
from utils import load_random_imgs
test_data_dir = 'img/shrine_temple/test/unknown'
x_test,true_labels = load_random_imgs(test_data_dir,seed=1)
x_test_preproc = preprocess_input(x_test.copy())/255.
probs = model.predict(x_test_preproc)
probs

ImportError: No module named 'utils'

### confirm the result

In [32]:
from utils import show_test_samples
show_test_samples(
    x_test,probs,
    img_itr_train.class_indices,
    true_labels
)

ImportError: No module named 'utils'